In [1]:
# Import required packages
import time
import os

import torch
from tqdm import tqdm

import wandb
from utils.model import *
from utils.robot import Robot
from utils.settings import config as cfg
from utils.utils import *
from pprint import pprint

In [2]:
def train_step(model, batch, optimizer, scheduler):
    """
    _summary_

    Args:
        model (_type_): _description_
        batch (_type_): _description_
        optimizer (_type_): _description_
        scheduler (_type_): _description_

    Returns:
        _type_: _description_
    """
    x, y = add_noise(batch)

    loss = -model(y).log_prob(x)  # -log p(x | y)
    loss = loss.mean()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    scheduler.step()

    return loss.item()

In [9]:
def mini_train(
    config=None,
    robot=None,
    J_tr=None, 
    P_tr=None, 
    J_ts=None, 
    P_ts=None, 
    F=None, 
    knn=None):
    # data generation
    if torch.cuda.is_available():
        device = 'cuda'
    else:
        device = 'cpu'
    
    train_loader = get_train_loader(
        J=J_tr, 
        P=P_tr, 
        F=F, 
        device=device, 
        batch_size=config["batch_size"])
    
    # Build Generative model, NSF
    # Neural spline flow (NSF) with 3 sample features and 5 context features
    solver, optimizer, scheduler = get_flow_model(
        load_model=cfg.use_pretrained,
        num_transforms=config["num_transforms"],
        subnet_width=config["subnet_width"],
        subnet_num_layers=config["subnet_num_layers"],
        lr=config["lr"],
        lr_weight_decay=config["lr_weight_decay"],
        decay_step_size=config["decay_step_size"],
        gamma=config["gamma"],
        device=device)
    
    solver.train()

    for ep in range(config["num_epochs"]):
        t = tqdm(train_loader)
        step = 0
        batch_loss = np.zeros((len(train_loader)))
        for batch in t:
            loss = train_step(
                model=solver, batch=batch, optimizer=optimizer, scheduler=scheduler
            )
            batch_loss[step] = loss
            bar = {"loss": f"{np.round(loss, 3)}"}
            t.set_postfix(bar, refresh=True)

            step += 1

        rand = np.random.randint(low=0, high=len(P_ts), size=cfg.num_eval_size)
        _, position_errors, _ = test(
            robot=robot,
            P_ts=P_ts[rand],
            F=F,
            solver=solver,
            knn=knn,
            K=cfg.K,
            print_report=False,
        )
        wandb.log({
            'epoch': ep, 
            'position_errors': position_errors.mean(),
            'train_loss': batch_loss.mean(), 
        })
        
    print("Finished Training")

In [14]:
sweep_config = {
    'name': 'sweep',
    'method': 'random',
    'metric' : {
        'name': 'position_errors',
        'goal': 'minimize'   
    },
    'parameters' : {
        'subnet_width': {
            'values': [128, 256, 512, 1024, 2048]
            },
        'subnet_num_layers': {
            'values': [int(i * 2) for i in range(2, 5)]
            },
        'num_transforms': {
            'values': [int(i * 2) for i in range(2, 10)]
        },
        'lr': {
            # a flat distribution between 0 and 0.1
            'distribution': 'q_uniform',
            'q': 1e-8,
            'min': 1e-8,
            'max': 1e-4,
        },
        'lr_weight_decay': {
            # a flat distribution between 0 and 0.1
            'distribution': 'q_uniform',
            'q': 1e-2,
            'min': 1e-2,
            'max': 9e-1,
        },
        'decay_step_size': {
            'values': [int(i * 2e3) for i in range(1, 20)],
        },
        'gamma': {
            'distribution': 'q_uniform',
            'q': 1e-2,
            'min': 1e-2,
            'max': 1,
        },
        'batch_size': {
            'values': [512, 1024, 2048]
        },
        'num_epochs': {
            'value': 5
        },
    },
}

In [15]:
sweep_id = wandb.sweep(
    sweep=sweep_config, 
    project='sdik_sweep',
    entity='luca_nthu')

Create sweep with ID: 0h4h1h61
Sweep URL: https://wandb.ai/luca_nthu/sdik_sweep/sweeps/0h4h1h61


In [16]:
robot = Robot(verbose=False)
J_tr, P_tr = data_collection(robot=robot, N=cfg.N_train)
J_ts, P_ts = data_collection(robot=robot, N=cfg.N_test)
F = posture_feature_extraction(J_tr)
knn = load_pickle(file_path=cfg.path_knn)

hnne load successfully from /home/luca/ikpflow/weights/panda/hnne.pickle


In [17]:
def main():
    # note that we define values from `wandb.config`  
    # instead of defining hard values
    run = wandb.init()

    # note that we define values from `wandb.config`  
    # instead of defining hard values
    config = {
        'subnet_width': wandb.config.subnet_width,
        'subnet_num_layers': wandb.config.subnet_num_layers,
        'num_transforms': wandb.config.num_transforms,
        'lr': wandb.config.lr,
        'lr_weight_decay': wandb.config.lr_weight_decay,
        'decay_step_size': wandb.config.decay_step_size,
        'gamma': wandb.config.gamma,
        'batch_size': wandb.config.batch_size,
        'num_epochs': wandb.config.num_epochs,
    }

    mini_train(
        config=config,
        robot=robot,
        J_tr=J_tr,
        P_tr=P_tr,
        J_ts=J_ts,
        P_ts=P_ts,
        F=F,
        knn=knn
    )


In [18]:
# Start sweep job.
wandb.agent(sweep_id, function=main, count=100)

wandb: Agent Starting Run: jtxk9sgi with config:
wandb: 	batch_size: 2048
wandb: 	decay_step_size: 14000
wandb: 	gamma: 0.89
wandb: 	lr: 2.035e-05
wandb: 	lr_weight_decay: 0.19
wandb: 	num_epochs: 5
wandb: 	num_transforms: 18
wandb: 	subnet_num_layers: 4
wandb: 	subnet_width: 256
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create a new model and start training.


100%|██████████| 1220/1220 [01:51<00:00, 10.97it/s, loss=2.712]


Finished Training


epoch,▁▃▅▆█
position_errors,█▃▃▂▁
train_loss,█▄▂▁▁
epoch,4
position_errors,0.23065
train_loss,2.98964


wandb: Agent Starting Run: 39s84yc1 with config:
wandb: 	batch_size: 1024
wandb: 	decay_step_size: 16000
wandb: 	gamma: 0.4
wandb: 	lr: 8.647e-05
wandb: 	lr_weight_decay: 0.6
wandb: 	num_epochs: 5
wandb: 	num_transforms: 16
wandb: 	subnet_num_layers: 4
wandb: 	subnet_width: 128
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
